In [1]:
import easygraph as eg
import csv
import pandas as pd
import networkx as nx

def make_graph(filepath:str):
    G=eg.Graph()
    nodes=[]
    nodes_attr=[]
    edges=[]
    edges_attr=[]
    f=open(filepath,'r')
    next(f)
    csv_reader=csv.reader(f)
    for row in csv_reader:
        i=str(row[0])
        i_affifiation=str(row[1])
        i_country=str(row[2])
        j=str(row[3])
        j_affifiation=str(row[4])
        j_country=str(row[5])
        year=int(row[6])
        area=str(row[7])

        nodes.append(i)
        temp1={}
        temp1['affiliation']=i_affifiation
        temp1['country']=i_country
        nodes_attr.append(temp1)

        nodes.append(j)
        temp2={}
        temp2['affiliation']=j_affifiation
        temp2['country']=j_country
        nodes_attr.append(temp2)

        edges.append((i,j))
        temp3={}
        temp3['year']=year
        temp3['area']=area
        edges_attr.append(temp3)

    G.add_nodes(nodes,nodes_attr)
    G.add_edges(edges,edges_attr)
    return G


In [2]:
# author
G1=make_graph('faculty-coauthors.csv')

In [3]:
SHS=sorted(eg.constraint(G1).items(), key=lambda d: d[1])

In [4]:
import random
con=eg.constraint(G1)
con=sorted(con.items(), key=lambda d: d[1])


SHS1=[]
nonSHS1=[]

kk=int(len(G1)*0.05)
k=0
for i in con:
    if k<kk:
        SHS1.append(i[0])
        k+=1
    else:
        break
        
random.seed(0)
while len(nonSHS1)<kk:
    t=random.sample(con, 1)
    if t[0][0] not in SHS1 and t[0][0] not in nonSHS1:
        nonSHS1.append(t[0][0])
        

In [5]:
print(SHS1)

['Fei-Yue Wang 0001', 'Tom A. B. Snijders', 'Johan H. Koskinen', 'Garry Robins', 'Xiao Wang 0002', 'Stephen P. Borgatti', 'Yong Yuan', 'Peng Wang 0002', 'Feng Xia 0001', 'Shui Yu', 'Qingpeng Zhang', 'Niloy Ganguly', 'Alessandro Lomi', 'Christian Steglich', 'Filip Agneessens', 'Philippa Pattison', 'Kathleen M. Carley', 'Gautam Srivastava', 'Weili Wu', 'V. S. Subrahmanian', 'Emmanuel Lazega', 'Jun Jason Zhang', 'Zhipeng Cai 0001', 'Ulrik Brandes', 'Laurence T. Yang', 'Christoph Stadtfeld', 'Giuseppe (Joe) Labianca', 'Juanjuan Li', 'Lianyong Qi', 'Patrick Doreian', 'Ding-Zhu Du', 'Dongpu Cao', 'Yisheng Lv', 'Carter T. Butts', 'Christopher McCarty', 'Shahid Mumtaz', 'Vincent Buskens', 'Carlo Morselli', 'Marijtje A. J. Van Duijn', 'Lifang Li', 'René Veenstra', 'Thomas W. Valente', 'Noshir S. Contractor', 'Bruce A. Desmarais', 'Galina Daraganova', 'Martin Bouchard 0002', 'Gang Xiong 0001', 'Yulei Wu', 'Shuangshuang Han', 'Rense Corten', 'Martin G. Everett', 'Pip Pattison', 'Jinyin Chen', 'Sh

In [ ]:
List={}
for i in range(0,len(SHS1)):
    List[SHS1[i]]={}
    